# Compare Model Hyperparameters

In [1]:
! git clone https://github.com/MaxHReinhardt/ArchitecturalStyleClassification

Cloning into 'ArchitecturalStyleClassification'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 129 (delta 63), reused 94 (delta 40), pack-reused 0
Receiving objects: 100% (129/129), 94.83 KiB | 15.80 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [2]:
import sys
import os

py_file_location = "/content/ArchitecturalStyleClassification/src"
sys.path.append(os.path.abspath(py_file_location))

In [3]:
import torch
import random
import numpy as np

from preprocessing import TrainSetDynamicNormalization, EvaluationSetDynamicNormalization
from model import MobileNetV1
from train_model import train_with_early_stopping
from evaluate_model import evaluate


def compare_training_hyperparameter_configurations(learning_rate_range, batch_size_range, num_configurations, train_csv, validation_csv):
    weight_decay = 0
    max_num_epochs = 100

    resolution = 384
    width_multiplier = 1
    cbam_last_layer = False
    cbam_all_layers = True

    # Check if CUDA (GPU) is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Device: {device}")

    for _ in range(num_configurations):
        # Randomly select learning rate and batch size from the provided ranges
        learning_rate = np.power(10, np.random.uniform(np.log10(learning_rate_range[0]), np.log10(learning_rate_range[1])))
        batch_size = np.power(2, np.random.uniform(np.log2(batch_size_range[0]), np.log2(batch_size_range[1])))

        # tiny datasets only for testing purposes
        train_set = TrainSetDynamicNormalization(resolution=resolution,
                                                 train_csv=train_csv).get_data()
        validation_set = EvaluationSetDynamicNormalization(resolution=resolution,
                                                           validation_csv=validation_csv).get_data()

        model = MobileNetV1(ch_in=3, n_classes=25, width_multiplier=width_multiplier,
                            cbam_last_layer=cbam_last_layer, cbam_all_layers=cbam_all_layers)
        model.to(device)

        trained_model, train_loss_development, val_loss_development = train_with_early_stopping(model,
                                                                                                train_set,
                                                                                                validation_set,
                                                                                                batch_size,
                                                                                                learning_rate,
                                                                                                max_num_epochs,
                                                                                                device,
                                                                                                weight_decay)

        model_name = f"{width_multiplier}-MobileNetV1-{resolution}_cbam_all_layers_lr-{learning_rate}_bs-{batch_size}"
        model_path = os.path.join("stored_models/", model_name + ".pth")
        torch.save(trained_model.state_dict(), model_path)

        accuracy, _, avg_loss, avg_prediction_time = evaluate(trained_model,
                                                              validation_set,
                                                              batch_size,
                                                              device)

        print(f"{model_name} evaluation -- Accuracy: {accuracy}, Average loss: {avg_loss}, "
              f"Average prediction time (seconds): {avg_prediction_time}")

In [4]:
!mkdir stored_models

In [5]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
!unzip /content/gdrive/MyDrive/data.zip;

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_305.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_113.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_107.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_106.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_112.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_304.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_310.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_37.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_338.jpg  
  inflating: data/dataset/train/Queen Anne architecture/Queen Anne architecture_23.jpg  
  inflating: data/dataset/train/Que

In [7]:
drive.flush_and_unmount()

## Run Experiment

In [8]:
learning_rate_range = [1e-5, 1e-2]
batch_size_range = [32, 128]
num_configurations = 10
train_csv = "data/dataset/train_annotation.csv"
validation_csv = "data/dataset/validation_annotation.csv"

In [9]:
compare_training_hyperparameter_configurations(learning_rate_range, batch_size_range, num_configurations, train_csv, validation_csv)

Device: cuda
Epoch: 1, train_loss: 3.021445, val_loss: 4.215442, Early stopping status: Initiated.
Epoch: 2, train_loss: 2.750767, val_loss: 2.806863, Early stopping status: Improvement found, counter reset to 0.
Epoch: 3, train_loss: 2.614229, val_loss: 3.093308, Early stopping status: No improvement in the last 1 epochs.
Epoch: 4, train_loss: 2.455756, val_loss: 2.503067, Early stopping status: Improvement found, counter reset to 0.
Epoch: 5, train_loss: 2.295140, val_loss: 2.787069, Early stopping status: No improvement in the last 1 epochs.
Epoch: 6, train_loss: 2.144660, val_loss: 2.515213, Early stopping status: No improvement in the last 2 epochs.
Epoch: 7, train_loss: 2.024909, val_loss: 2.102790, Early stopping status: Improvement found, counter reset to 0.
Epoch: 8, train_loss: 1.895017, val_loss: 2.209698, Early stopping status: No improvement in the last 1 epochs.
Epoch: 9, train_loss: 1.820625, val_loss: 2.348607, Early stopping status: No improvement in the last 2 epochs.